In [1]:
import os
import glob
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import plotly.express as px
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report
from PIL import Image
import io
import base64
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import confusion_matrix
import gc

from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAvgPool2D,Conv2D, MaxPool2D, BatchNormalization, Dropout, Input, LeakyReLU, Reshape, Conv2DTranspose, Concatenate, Lambda, MaxPooling2D
from tensorflow.keras.initializers import GlorotUniform
 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model


In [2]:
# Kaggle API Token
from google.colab import files
files.upload()

# Setup Kaggle API
!pip install kaggle --quiet
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the Dataset
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
 # Extract the Dataset
!unzip -q chest-xray-pneumonia.zip

Saving kaggle.json to kaggle.json
100% 2.29G/2.29G [01:46<00:00, 23.5MB/s]
100% 2.29G/2.29G [01:46<00:00, 23.2MB/s]


In [ ]:
# Set up the directory path for training and testing images
base_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray'

In [ ]:
import os
import shutil

# Set up the directory path for training and testing images
base_dir = './chest_xray'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Create directories to combine images in the training and testing directories
combined_dir = os.path.join(base_dir, 'combined')
os.makedirs(combined_dir, exist_ok=True)

# Create new 'NORMAL' and 'PNEUMONIA' directories inside the combined directory
normal_dir = os.path.join(combined_dir, 'NORMAL')
pneumonia_dir = os.path.join(combined_dir, 'PNEUMONIA')
os.makedirs(normal_dir, exist_ok=True)
os.makedirs(pneumonia_dir, exist_ok=True)

# Function to copy files from source directories to a target directory
def copy_files(source_dir, target_dir):
   count = 0
   for filename in os.listdir(source_dir):
      file_path = os.path.join(source_dir, filename)
      if os.path.isfile(file_path):
         count += 1
         shutil.copy(file_path, target_dir)

   print(f'Copied {count} files from {source_dir} to {target_dir}')
# Copy all 'NORMAL' images from the training, validation, and test directories to the new 'NORMAL' directory
copy_files(os.path.join(train_dir, 'NORMAL'), normal_dir)
copy_files(os.path.join(val_dir, 'NORMAL'), normal_dir)
copy_files(os.path.join(test_dir, 'NORMAL'), normal_dir)

# Copy all 'PNEUMONIA' images from the training, validation, and test directories to the new 'PNEUMONIA' directory
copy_files(os.path.join(train_dir, 'PNEUMONIA'), pneumonia_dir)
copy_files(os.path.join(val_dir, 'PNEUMONIA'), pneumonia_dir)
copy_files(os.path.join(test_dir, 'PNEUMONIA'), pneumonia_dir)


In [ ]:
# Plot the overall distribution of the training, validation, and test datasets using plotly

# Create a DataFrame with the number of images in each category
data = {
    'Dataset': ['Training', 'Validation', 'Test', 'All'],
    'NORMAL': [len(os.listdir(os.path.join(train_dir, 'NORMAL'))), len(os.listdir(os.path.join(val_dir, 'NORMAL'))), len(os.listdir(os.path.join(test_dir, 'NORMAL'))), len(os.listdir(normal_dir))],
    'PNEUMONIA': [len(os.listdir(os.path.join(train_dir, 'PNEUMONIA'))), len(os.listdir(os.path.join(val_dir, 'PNEUMONIA'))), len(os.listdir(os.path.join(test_dir, 'PNEUMONIA'))), len(os.listdir(pneumonia_dir))]
}

# Plot the distribution of images in each category
fig = px.bar(pd.DataFrame(data).melt(id_vars='Dataset'), x='Dataset', y='value', color='variable', barmode='group')
fig.update_layout(title='Distribution of Images in Each Category', xaxis_title='Dataset', yaxis_title='Number of Images')
fig.show()





In [ ]:
'''
# Preprocessing the images - setting up image data generators
batch_size = 24
image_size = 224
color_ch = 3

# Limited data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20,
                     width_shift_range=0.2, height_shift_range=0.2)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
combined_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size),
                                       batch_size=batch_size, class_mode='binary')
val_generator = val_datagen.flow_from_directory(test_dir, target_size=(image_size, image_size),
                                     batch_size=batch_size, class_mode='binary')
test_generator = test_datagen.flow_from_directory(val_dir, target_size=(image_size, image_size),
                                      batch_size=batch_size, class_mode='binary')
combined_generator = combined_datagen.flow_from_directory(combined_dir, target_size=(image_size, image_size),
                                       batch_size=batch_size, class_mode='binary')


# Use tf.data.Dataset for efficient data loading
def ae_wrapper(generator):
   for x_batch, _ in generator:
      #x_batch = tf.image.rgb_to_grayscale(x_batch) # Convert images to grayscale
      x_batch = tf.cast(x_batch, tf.float32) # Convert images to float32
      yield (x_batch, x_batch)

def clf_wrapper(generator):
   for x_batch, y_batch in generator:
      #x_batch = tf.image.rgb_to_grayscale(x_batch)  # Convert images to grayscale
      x_batch = tf.cast(x_batch, tf.float32) # Convert images to float32
      yield (x_batch, y_batch)

def create_dataset(generator, wrapperFunc, output, buffer_size=50):
   return tf.data.Dataset.from_generator(
      lambda: wrapperFunc(generator),
      output_signature=output
   ).shuffle(buffer_size).prefetch(tf.data.AUTOTUNE)

# Set the buffer size for shuffling the dataset
buffer_size = 50
# Define the output signature for the datasets (the ae and the clf have different signatures)
ae_ds_output = (tf.TensorSpec(shape=(image_size, image_size, color_ch), dtype=tf.float32), tf.TensorSpec(shape=(image_size, image_size, color_ch), dtype=tf.float32))
clf_ds_output = (tf.TensorSpec(shape=(image_size, image_size, color_ch), dtype=tf.float32), tf.TensorSpec(shape=(), dtype=tf.float32))

ae_combined = tf.data.Dataset.from_generator(
   lambda: ae_wrapper(combined_generator),
   output_signature=ae_ds_output
).shuffle(buffer_size).prefetch(tf.data.AUTOTUNE)


clf_train = create_dataset(
   train_generator,
   clf_wrapper,
   clf_ds_output,
   buffer_size)

clf_val = create_dataset(
   val_generator,
   clf_wrapper,
   clf_ds_output,
   buffer_size)

clf_test = create_dataset(
   test_generator,
   clf_wrapper,
   clf_ds_output,
   buffer_size)

print(ae_combined.element_spec)
print(clf_train.element_spec)

'''

In [ ]:
del train_generator
del val_generator
del test_generator
del combined_generator
del train_dataset
del val_dataset
del test_dataset
del combined_dataset
del train_datagen
del val_datagen
del test_datagen
del combined_datagen




gc.collect()

In [4]:

# Preprocessing the images - setting up image data generators
batch_size = 24
image_size = 224

# Limited data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20,
                           width_shift_range=0.2, height_shift_range=0.2)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
combined_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size),
                                       batch_size=batch_size, class_mode='binary')
val_generator = val_datagen.flow_from_directory(test_dir, target_size=(image_size, image_size),
                                     batch_size=batch_size, class_mode='binary')
test_generator = test_datagen.flow_from_directory(val_dir, target_size=(image_size, image_size),
                                      batch_size=batch_size, class_mode='binary')
combined_generator = combined_datagen.flow_from_directory(combined_dir, target_size=(image_size, image_size),
                                       batch_size=batch_size, class_mode='binary')


# Use tf.data.Dataset for efficient data loading
def generator_wrapper(generator):
   for x_batch, _ in generator:
      x_batch = tf.cast(x_batch, tf.float32)
      yield (x_batch, x_batch)
# Shuffle buffer size
buffer_size = 50

train_dataset = tf.data.Dataset.from_generator(
   lambda: generator_wrapper(train_generator),
   output_signature=(
      tf.TensorSpec(shape=(None, image_size, image_size, 3), dtype=tf.float32),
      tf.TensorSpec(shape=(None, image_size, image_size, 3), dtype=tf.float32))
).shuffle(buffer_size).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_generator(
   lambda: generator_wrapper(val_generator),
   output_signature=(
      tf.TensorSpec(shape=(None, image_size, image_size, 3), dtype=tf.float32),
      tf.TensorSpec(shape=(None, image_size, image_size, 3), dtype=tf.float32))
).shuffle(buffer_size).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(
   lambda: generator_wrapper(test_generator),
   output_signature=(
      tf.TensorSpec(shape=(None, image_size, image_size, 3), dtype=tf.float32),
      tf.TensorSpec(shape=(None, image_size, image_size, 3), dtype=tf.float32))
).shuffle(buffer_size).prefetch(tf.data.AUTOTUNE)

combined_dataset = tf.data.Dataset.from_generator(
   lambda: generator_wrapper(combined_generator),
   output_signature=(
      tf.TensorSpec(shape=(None, image_size, image_size, 3), dtype=tf.float32),
      tf.TensorSpec(shape=(None, image_size, image_size, 3), dtype=tf.float32))
).shuffle(buffer_size).prefetch(tf.data.AUTOTUNE)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
img = next(train_generator)[0][0]
plt.imshow(img)
img = tf.image.rgb_to_grayscale(img)
plt.imshow(img[:, :, 0], cmap='gray')

In [13]:
# Define the callbacks

ae_early_stopping = EarlyStopping(monitor='val_mean_squared_error', patience=3, verbose=1, mode='min', restore_best_weights=True)
ae_model_checkpoint = ModelCheckpoint('conv_ae.keras', monitor='val_mean_squared_error', verbose=1,mode='min', save_best_only=True)
ae_reduce_lr = ReduceLROnPlateau(monitor='val_mean_squared_error', factor=0.01, patience=4, verbose=1, mode='min')

# Define the callbacks for the classifier (monitor validation accuracy)
classifier_early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1, mode='max', restore_best_weights=True)
classifier_model_checkpoint = ModelCheckpoint('enhanced_clf_v1.keras', monitor='val_accuracy', verbose=1, mode='max', save_best_only=True)
classifier_reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.01, patience=4, verbose=1, mode='max')

# Convolutional Autoencoder

In [ ]:

# Encoder
conv_encoder = Sequential([
    Reshape([image_size, image_size, 3], name="input"), # input: 224x224x1
    BatchNormalization(),
    Conv2D(16, 3, padding="same", activation="relu"), 
    MaxPool2D(pool_size=2), # output: 112x112x16
    Dropout(0.15),
    BatchNormalization(),
    Conv2D(32, 3, padding="same", activation="relu"),
    MaxPool2D(pool_size=2), # output: 56x56x32
    Dropout(0.35),
    BatchNormalization(),
    Conv2D(64, 3, padding="same", activation="relu"),
    MaxPool2D(pool_size=2), # output: 28x28x64
    Dropout(0.55),
    BatchNormalization(),
    Conv2D(128, 5, padding="same", activation="relu"),
    MaxPool2D(pool_size=2), # output: 14x14x128
    Dropout(0.35),
    BatchNormalization(),
    Conv2D(256, 5, padding="same", activation="relu"),
    MaxPool2D(pool_size=2), # output: 7x7x256
    Dropout(0.15),
    BatchNormalization(),
    Conv2D(512, 5, padding="same", activation="relu"),
    MaxPool2D(pool_size=2), # output: 3x3x512
    Dropout(0.15),
    BatchNormalization(),
    Conv2D(60, 3, padding="same", activation="relu"),
    GlobalAvgPool2D() # output: 60
])

# Decoder
conv_decoder = Sequential([
    Dense(28 * 28 * 512),
    Reshape((28, 28, 512), name="decoder_input"), # input: 28x28x512
    Conv2DTranspose(512, 5, strides=2, padding="same", activation="relu"), # output: 56x56x512
    Dropout(0.5),
    BatchNormalization(),
    Conv2DTranspose(256, 5, strides=2, padding="same", activation="relu"), # output: 112x112x256
    Dropout(0.25),
    BatchNormalization(),
    Conv2DTranspose(128, 5, strides=2, padding="same", activation="relu"), # output: 224x224x128
    Dropout(0.5),
    BatchNormalization(),
    Conv2DTranspose(64, 3, strides=1, padding="same", activation="relu"), # output: 224x224x64
    Dropout(0.25),
    BatchNormalization(),
    Conv2DTranspose(32, 3, strides=1, padding="same", activation="relu"), # output: 224x224x32
    Dropout(0.15),
    BatchNormalization(),
    Conv2DTranspose(16, 3, strides=1, padding="same", activation="relu"), # output: 224x224x16
    Dropout(0.05),
    BatchNormalization(),
    Conv2DTranspose(3, 3, strides=1, padding="same", activation="sigmoid"), # output: 224x224x1
    Reshape([image_size, image_size, 3], name="output")
])

# Autoencoder
conv_ae = Sequential([conv_encoder, conv_decoder])


conv_ae.compile(
    loss="mse", 
    # Use nadam with a 0.01 learning rate
    optimizer=tf.keras.optimizers.Nadam(learning_rate=0.1),
     metrics=[tf.keras.metrics.MeanSquaredError(),'accuracy'])

conv_ae.build(input_shape=(None, image_size, image_size, 3))
conv_ae.summary()



In [ ]:
tf.debugging.disable_traceback_filtering()
# reset keras backend
tf.keras.backend.clear_session()

In [ ]:
# Create a function to build and fit models with different bottleneck sizes
def build_and_fit_ae(bottleneck_range=[60, 200], bottleneck_step=20):
   for i in range(bottleneck_range[0], bottleneck_range[1], bottleneck_step):
      # Set a seed
      tf.random.set_seed(1)
      print(f"Training model with bottleneck size {i}")
      tf.keras.backend.clear_session()
      # Delete the model
      if 'ae' in locals():
         print('Deleting AE')
         del ae
      # Clear the cache
      gc.collect()

      # Encoder
      encoder = Sequential([
         Reshape([image_size, image_size, 3], name="input"), # input: 224x224x1
         BatchNormalization(),
         Conv2D(16, 3, padding="same", activation="relu"), 
         MaxPool2D(pool_size=2), # output: 112x112x16
         Dropout(0.15),
         BatchNormalization(),
         Conv2D(32, 3, padding="same", activation="relu"),
         MaxPool2D(pool_size=2), # output: 56x56x32
         Dropout(0.35),
         BatchNormalization(),
         Conv2D(64, 3, padding="same", activation="relu"),
         MaxPool2D(pool_size=2), # output: 28x28x64
         Dropout(0.55),
         BatchNormalization(),
         Conv2D(128, 5, padding="same", activation="relu"),
         MaxPool2D(pool_size=2), # output: 14x14x128
         Dropout(0.35),
         BatchNormalization(),
         Conv2D(256, 5, padding="same", activation="relu"),
         MaxPool2D(pool_size=2), # output: 7x7x256
         Dropout(0.15),
         BatchNormalization(),
         Conv2D(512, 5, padding="same", activation="relu"),
         MaxPool2D(pool_size=2), # output: 3x3x512
         Dropout(0.15),
         BatchNormalization(),
      ])

      # Decoder
      decoder = Sequential([
         Dense(28 * 28 * 512),
         Reshape((28, 28, 512), name="decoder_input"), # input: 28x28x512
         Conv2DTranspose(512, 5, strides=2, padding="same", activation="relu"), # output: 56x56x512
         Dropout(0.5),
         BatchNormalization(),
         Conv2DTranspose(256, 5, strides=2, padding="same", activation="relu"), # output: 112x112x256
         Dropout(0.25),
         BatchNormalization(),
         Conv2DTranspose(128, 5, strides=2, padding="same", activation="relu"), # output: 224x224x128
         Dropout(0.5),
         BatchNormalization(),
         Conv2DTranspose(64, 3, strides=1, padding="same", activation="relu"), # output: 224x224x64
         Dropout(0.25),
         BatchNormalization(),
         Conv2DTranspose(32, 3, strides=1, padding="same", activation="relu"), # output: 224x224x32
         Dropout(0.15),
         BatchNormalization(),
         Conv2DTranspose(16, 3, strides=1, padding="same", activation="relu"), # output: 224x224x16
         Dropout(0.05),
         BatchNormalization(),
         Conv2DTranspose(3, 3, strides=1, padding="same", activation="sigmoid"), # output: 224x224x1
         Reshape([image_size, image_size, 3], name="output")
      ])

      encoder.add(Conv2D(i, 3, padding="same", activation="relu", kernel_initializer=GlorotUniform(seed=i)))
      encoder.add(GlobalAvgPool2D())
      
      ae = Sequential([encoder, decoder], name=f"conv_ae_BN{i}")
      ae.compile(
         loss="mse", 
         optimizer=tf.keras.optimizers.Nadam(learning_rate=0.05),
         metrics=[tf.keras.metrics.MeanSquaredError(),'accuracy'])
      
      ae.build(input_shape=(None, image_size, image_size, 3))
  
    
      histories = []
      
      history = ae.fit(
         combined_dataset,
         epochs=5,
         steps_per_epoch=len(combined_generator)//10,
      )
      print('-----------------------------------------------------------------')
      histories.append(history)
   return histories

bottleneck_test = build_and_fit_ae(bottleneck_range=[30, 110], bottleneck_step=10)

The MNIST AE example from the Hands-On Machine Learning text reduced the input rank from 28x28x1 = 784 to 30, a 96.17% reduction. We are attempting to reduce our input rank from 224x224x3 = 150,528 to around 100, a 99.93% reduction. To make that difference more apparent, if we were to reduce rank by 96.17% on our input, we would have a bottleneck tensor rank of 5765 (150,528 * 0.0383). This is a significant difference in the amount of information we are trying to compress.

In [ ]:

# Plot the histories of the autoencoder models on the same plot (superimposed) using plotly
fig = go.Figure()
bottleneck_sizes = np.arange(30, 110, 10)
# Find the loss of the models at the final epoch
min_loss = np.inf
min_loss_idx = 0
max_acc = 0
max_acc_idx = 0
for i, history in enumerate(bottleneck_test):
   if history.history['loss'][-1] < min_loss:
      min_loss = history.history['loss'][-1]
      min_loss_idx = i
   if history.history['accuracy'][-1] > max_acc:
      max_acc = history.history['accuracy'][-1]
      max_acc_idx = i


for i, history in enumerate(bottleneck_test):
   fig.add_trace(go.Scatter(x=np.arange(1, 6), y=history.history['loss'], mode='lines', name=f'Filters {bottleneck_sizes[i]}'))
fig.update_layout(title='Autoencoder Training Loss for Different Bottleneck Sizes (Batch size: 24, steps-per-epoch: 24 [1/10th of all images])', xaxis_title='Epoch', yaxis_title='Loss')
# Add an annotation to show the best model
fig.add_annotation(x=5, ax=-100, y=min_loss, ay=-100, text=f'Min loss: {np.round(min_loss, 4)} @ {bottleneck_sizes[min_loss_idx]} filters', showarrow=True, arrowhead=1, font=dict(family="Courier New, monospace", size=16, color="#ffffff"), bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="black", opacity=0.8)
fig.show()

fig2 = go.Figure()
for i, history in enumerate(bottleneck_test):
   fig2.add_trace(go.Scatter(x=np.arange(1, 6), y=history.history['accuracy'], mode='lines', name=f'Filters {bottleneck_sizes[i]}'))
fig2.update_layout(title='Autoencoder Training Accuracy for Different Bottleneck Sizes (Batch size: 24, steps-per-epoch: 24 [1/10th of all images])', xaxis_title='Epoch', yaxis_title='Accuracy')
fig2.add_annotation(x=5, ax=-100, y=max_acc, ay=-20, text=f'Max accuracy: {np.round(max_acc, 4)} @ {bottleneck_sizes[max_acc_idx]} filters', showarrow=True, arrowhead=1, font=dict(family="Courier New, monospace", size=16, color="#ffffff"), bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="black", opacity=0.8)
fig2.show()

# Plot the variances in the loss for each model using a bar chart
loss_variances = [np.var(history.history['loss']) for history in bottleneck_test]
fig3 = go.Figure()
fig3.add_trace(go.Bar(x=bottleneck_sizes, y=loss_variances, name='Loss Variance'))
fig3.update_layout(title='Variance in Loss for Different Bottleneck Sizes (Batch size: 24, steps-per-epoch: 24 [1/10th of all images])', xaxis_title='Bottleneck Size (filters)', yaxis_title='Variance')
# Annotate the top of the highest and lowest bar with their variances
fig3.add_annotation(x=bottleneck_sizes[np.argmax(loss_variances)], y=np.max(loss_variances), text=f'{np.round(np.max(loss_variances), 4)}', showarrow=False, arrowhead=1, font=dict(family="Courier New, monospace", size=16, color="#ffffff"), bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="black", opacity=0.8)
fig3.add_annotation(x=bottleneck_sizes[np.argmin(loss_variances)], y=np.min(loss_variances), text=f'{np.round(np.min(loss_variances), 4)}', showarrow=False, arrowhead=1, font=dict(family="Courier New, monospace", size=16, color="#ffffff"), bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="black", opacity=0.8)
fig3.show()

# Plot the variances in the accuracy for each model using a bar chart
accuracy_variances = [np.var(history.history['accuracy']) for history in bottleneck_test]
fig4 = go.Figure()
fig4.add_trace(go.Bar(x=bottleneck_sizes, y=accuracy_variances, name='Accuracy Variance'))
fig4.update_layout(title='Variance in Accuracy for Different Bottleneck Sizes (Batch size: 24, steps-per-epoch: 24 [1/10th of all images])', xaxis_title='Bottleneck Size (filters)', yaxis_title='Variance')
# Annotate the top of the highest and lowest bar with their variances
fig4.add_annotation(x=bottleneck_sizes[np.argmax(accuracy_variances)], y=np.max(accuracy_variances), text=f'{np.round(np.max(accuracy_variances), 4)}', showarrow=False, arrowhead=1, font=dict(family="Courier New, monospace", size=16, color="#ffffff"), bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="black", opacity=0.8)
fig4.add_annotation(x=bottleneck_sizes[np.argmin(accuracy_variances)], y=np.min(accuracy_variances), text=f'{np.round(np.min(accuracy_variances), 4)}', showarrow=False, arrowhead=1, font=dict(family="Courier New, monospace", size=16, color="#ffffff"), bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="black", opacity=0.8)
fig4.show()

In [ ]:
# Train the autoencoder
history = conv_ae.fit(
   combined_dataset,
   epochs=30,
   validation_data=val_dataset,
   steps_per_epoch=(len(train_generator) + len(val_generator)),
   validation_steps=len(val_generator),
   callbacks=[
      ae_early_stopping,
      ae_model_checkpoint,
      ae_reduce_lr
   ])

In [ ]:
# Plot the training history using plotly

def plot_history(history):
   # Plot the mean squared error and accuracy
   fig = px.line(
      history.history,
      y=['mean_squared_error', 'accuracy'],
      labels={'index': 'epoch', 'value': 'mean squared error'},
      title='Autoencoder Training History'
   )
   fig.show()

plot_history(history)


In [ ]:
# Function to visualize the encoded images
def visualize(img, encoder, decoder):
   encoded = encoder.predict(img[None])[0]
   decoded = decoder.predict(encoded[None])[0]
   plt.figure(figsize=(10, 5))
   plt.subplot(1, 3, 1)
   plt.title("Original")
   plt.imshow(img)
   plt.axis('off')
   plt.subplot(1, 3, 2)
   plt.title("Encoded")
   plt.plot(encoded)
   plt.axis('off')
   plt.subplot(1, 3, 3)
   plt.title("Decoded")
   plt.imshow(decoded)
   plt.axis('off')
   plt.show()

# Visualize the images from the val_generator
for i in range(10):
   img = next(val_generator)[0][0]
   visualize(img, conv_encoder, conv_decoder)



# Enhanced Pneumonia Detection using Convolutional Autoencoder

In [ ]:
# Build the classifier as a Functional model, including a block of convolutional layers, then the encoder, then a block of dense layers
def build_classifier(encoder, weights={'vanilla': 0.5, 'encoder': 0.5}):
   # Freeze the encoder layers
   for layer in encoder.layers:
      layer.trainable = False

   # Create an input layer that contains two inputs: the encoder input and a shallow input
   input_= Input((image_size, image_size, 3), name='input')


   # Functional model, including a block of convolutional layers in tandem with the encoder (add one fully connected layer to learn the features anew), then a block of dense layers
   # Two paths: one for the vanilla input, one for the encoder output

   # Vanilla input
   x = Conv2D(16, (1, 1), strides=1, activation='relu', name='block_1_conv_1')(input_) # Output: 224x224x16
   x = Dropout(0.25, name='block_1_dropout')(x)
   x = BatchNormalization(name='block_1_batchNorm_1')(x)
   x = Conv2D(32, (3, 3), strides=2, activation='relu', name='block_1_conv_2')(x) # Output: 112x112x32
   x = Dropout(0.35, name='block_1_dropout_2')(x)
   x = BatchNormalization(name='block_1_batchNorm_2')(x)

   x = Conv2D(64, (3, 3), activation='relu', name='block_2_conv_1')(x) # Output: 112x112x64 
   x = Dropout(0.5, name='block_2_dropout_1')(x) 
   x = BatchNormalization(name='block_2_batchNorm_1')(x)    
   x = Conv2D(128, (3, 3), strides=2, activation='relu', name='block_2_conv_2')(x) # Output: 56x56x128
   x = Dropout(0.35, name='block_2_dropout_2')(x)
   x = BatchNormalization(name='block_2_batchNorm_2')(x)

   x = Conv2D(256, (5, 5), activation='relu', name='block_3_conv_1')(x) # Output: 56x56x256
   x = Dropout(0.15, name='block_3_dropout_1')(x)
   x = BatchNormalization(name='block_3_batchNorm_1')(x)   
   x = Conv2D(512, (5, 5), strides=2, activation='relu', name='block_3_conv_2')(x) # Output: 28x28x512
   x = Dropout(0.05, name='block_3_dropout_2')(x)
   x = BatchNormalization(name='block_3_batchNorm_2')(x)   
   x = GlobalAvgPool2D(name="block_4_globalAvgPool2D")(x) # Output: 512
   # Weight the vanilla output
   vanilla_output = Lambda(lambda x: x * weights['vanilla'], name="vanilla_weighted_output")(x) # Output: 512

   # Encoder 
   encoded_features = encoder(input_)
   x = BatchNormalization(name="encoder_batchNorm_1")(encoded_features)
   # Weight the encoder output
   encoder_output = Lambda(lambda x: x * weights['encoder'], name="encoder_weighted_output")(x) # Output: 60
 
   # average the two outputs
   DNN_output = layers.Average(name='average')([vanilla_output, encoder_output])

   # Create the model
   model = Model(inputs=input_, outputs=DNN_output)

   return model

# Build the classifier
van_weight = 0.9
enc_weight = 0.1
classifier = build_classifier(conv_encoder, weights={'vanilla': 0.8, 'encoder': 0.2})
classifier.compile(
   loss='binary_crossentropy',
   optimizer=tf.keras.optimizers.Nadam(learning_rate=0.01),
   metrics=['accuracy', 'AUC']
)

# SHow the block diagram of the classifier using plot_model
plot_model(classifier, show_shapes=True, show_layer_names=True)


### Concatenate the weighted inputs

   combined_output = Concatenate(name="combined_output")([vanilla_output, encoder_output])
   x = BatchNormalization(name="combined_batchNorm_1")(combined_output)
   x = Dense(256, activation='relu', kernel_initializer="he_normal", name="combined_dense_1")(x)
   x = Dropout(0.5, name="combined_dropout_1")(x)
   x = BatchNormalization(name="combined_batchNorm_2")(x)
   x = Dense(128, activation='relu', kernel_initializer="he_normal", name="combined_dense_2")(x)
   x = Dropout(0.3, name="combined_dropout_2")(x)
   x = BatchNormalization(name="combined_batchNorm_3")(x)
   x = Dense(64, activation='relu', kernel_initializer="he_normal", name="combined_dense_3")(x)
   x = Dropout(0.1, name="combined_dropout_3")(x)
   x = BatchNormalization(name="combined_batchNorm_4")(x)
   x = Dense(32, activation='relu', kernel_initializer="he_normal", name="combined_dense_4")(x)
   x = Dropout(0.1, name="combined_dropout_4")(x)
   x = BatchNormalization(name="combined_batchNorm_5")(x)


In [ ]:
# Function to build and fit models with different weights for the vanilla and encoder inputs

def build_and_fit_classifier():
   # For loop to train and evaluate different weight splits
   iterations=11
   scores = []
   for i in range(iterations):
      print(f"Training model {i+1}/{iterations}")
      # Define the weights
      weights = {
         'vanilla': 0.1 * i,
         'encoder': 1 - (0.1 * i)
      }
      print(f'Weights: {weights}')

      # Build the classifier
      classifier = build_classifier(conv_encoder, weights=weights)
      classifier.compile(
         loss='binary_crossentropy',
         optimizer=tf.keras.optimizers.Nadam(learning_rate=0.01),
         metrics=['accuracy', 'AUC']
      )

      # Train the classifier
      history = classifier.fit(
         train_generator,
         epochs=10,
         validation_data=val_generator,
         steps_per_epoch=len(train_generator)//10,
         validation_steps=len(val_generator)//10,
         callbacks=[
            classifier_early_stopping,
            classifier_model_checkpoint,
            classifier_reduce_lr
         ]
      )

      # Evaluate the classifier
      scores.append(classifier.evaluate(test_generator, return_dict=True))
      
   return scores, history

# Build and fit the classifier
scores,best_history = build_and_fit_classifier()

# Plot the highest scores using plotly

def plot_scores(scores):
      # Plot the accuracy and AUC
      fig = px.bar(
         scores,
         y=['accuracy', 'auc'],
         labels={'index': 'model', 'value': 'score'},
         title='Classifier Scores'
      )
      # annotate the weights for each model
      fig.update_layout(
         xaxis=dict(tickvals=list(range(len(scores))),
                     ticktext=[f'Weights: 0.{i} / 0.{10-i}' for i in range(11)]
         )
      )
      fig.show()

plot_scores(scores)

In [ ]:
# Train the classifier
history = classifier.fit(
   # train on just 100 images
   train_generator,
   validation_data=val_generator,
   epochs=15,
   steps_per_epoch=len(train_generator)//4,
   validation_steps=len(val_generator)//4,
   callbacks=[classifier_early_stopping, classifier_model_checkpoint, classifier_reduce_lr]
)

In [ ]:
# Evaluate the classifier
#classifier.evaluate(val_generator, steps=len(val_generator))

# Get the true labels
y_true = val_generator.classes
# Get the predicted labels
y_pred = classifier.predict(val_generator).argmax(axis=1)
# Round the predicted labels up and down
y_pred = np.round(y_pred)
# Print the classification report
print(classification_report(y_true, y_pred, target_names=val_generator.class_indices.keys()))

# Plot the training history using plotly
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['accuracy'], mode='lines', name='Train'))
fig.add_trace(go.Scatter(y=history.history['val_accuracy'], mode='lines', name='Validation'))
fig.add_trace(go.Scatter(y=history.history['auc'], mode='lines', name='AUC'))
fig.update_layout(title=f'Accuracy<br> (vanilla path: {van_weight})', xaxis_title='Epochs', yaxis_title='Accuracy')
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Train'))
fig.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name='Validation'))
fig.update_layout(title='Loss', xaxis_title='Epochs', yaxis_title='Loss')
fig.show()


In [ ]:
# Print the predictions images and labels from the classifier
predictions = classifier.predict(val_generator, steps=len(val_generator))
example_true_labels = val_generator.classes
for i in range(5):
   img = next(val_generator)[0][0]
   plt.imshow(img)
   plt.title(f'Prediction: {predictions[i]}, True Label: {example_true_labels[i]}')

   plt.show()

# Denoising Autoencoder

In [ ]:
tf.random.set_seed(42)

denoising_encoder = tf.keras.Sequential([
   tf.keras.layers.GaussianNoise(0.1),
   tf.keras.layers.Conv2D(30, 3, padding="same", activation="relu"),
   tf.keras.layers.MaxPool2D(),
   tf.keras.layers.Flatten(),
   tf.keras.layers.Dense(512, activation="relu"),
])
denoising_decoder = tf.keras.Sequential([
   tf.keras.layers.Dense(79 * 79 * 3, activation="relu"),  # Adjusted for 224x224x3 images
   tf.keras.layers.Reshape([79, 79, 3]),  # Adjusted for 224x224x3 images
   tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3, strides=2,
                         padding="same", activation="sigmoid")
])

denoising_ae = tf.keras.Sequential([denoising_encoder, denoising_decoder])
denoising_ae.compile(loss="binary_crossentropy", optimizer="nadam",
                metrics=["mse",  "accuracy"])
history = denoising_ae.fit(train_dataset, 
                           epochs=10,
                           validation_data=test_dataset,
                           callbacks=[early_stopping, model_checkpoint, reduce_lr],
                           verbose=1,
                           steps_per_epoch=len(train_generator),
                           validation_steps=len(test_generator))

In [ ]:
n_images = 5
new_images = val_dataset.take(n_images)
# Get the image data for the first image
new_images = next(iter(new_images))[0]
# Add some noise to the images
new_images_noisy = new_images + np.random.normal(loc=0.0, scale=0.1, size=new_images.shape)
new_images_denoised = denoising_ae.predict(new_images_noisy)

# Plot the images
plt.figure(figsize=(20, 6))
for i in range(n_images):
   plt.subplot(3, n_images, i + 1)
   plt.imshow(new_images[i])
   plt.axis('off')
   plt.subplot(3, n_images, i + 1 + n_images)
   plt.imshow(new_images_noisy[i])
   plt.axis('off')
   plt.subplot(3, n_images, i + 1 + 2 * n_images)
   plt.imshow(new_images_denoised[i])
   plt.axis('off')

plt.show()
